<a href="https://colab.research.google.com/github/CarlosEstellita/Atlantico-Academy-Bootcamp/blob/main/1.%20Activities/3_Vis%C3%A3o_Computacional_trilha_avan%C3%A7ada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IA BOOTCAMP - 3. Visão Computacional trilha avançada

O Dataset foi importado via Google Drive!

##Seção Única

###Importando bibliotecas

In [ ]:
%matplotlib inline  
import warnings 
warnings.filterwarnings('always') 
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
import cv2 as cv
import os 
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.utils import to_categorical
from keras.layers import Dropout, Flatten, Input, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

###Lendo e tratando os dados

In [ ]:
daisy_dir = glob.glob(os.path.join('drive/MyDrive/flowers/daisy/', '*'))
dandelion_dir = glob.glob(os.path.join('drive/MyDrive/flowers/dandelion/', '*'))
rose_dir = glob.glob(os.path.join('drive/MyDrive/flowers/rose/', '*'))
sunflower_dir = glob.glob(os.path.join('drive/MyDrive/flowers/sunflower/', '*'))
tulip_dir = glob.glob(os.path.join('drive/MyDrive/flowers/tulip/', '*'))

X_path = daisy_dir + dandelion_dir + rose_dir + sunflower_dir + tulip_dir

X = []

for f in X_path:
  X.append(np.array(cv.resize(cv.imread(f), (224,224), interpolation = cv.INTER_AREA))) 
X = np.array(X)
X = X / 255

###Definindo as tabelas

In [ ]:
l_daisy = np.zeros(len(daisy_dir))
l_dandelion = np.ones(len(dandelion_dir))
l_rose = 2*np.ones(len(rose_dir))
l_sunflower = 3*np.ones(len(sunflower_dir))
l_tulip = 4*np.ones(len(tulip_dir))

y = np.concatenate((l_daisy, l_dandelion, l_rose, l_sunflower, l_tulip))
y = to_categorical(y, 5)

###Separando e treinando validação

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=42)

In [ ]:
del X
del y
del daisy_dir
del dandelion_dir
del tulip_dir
del rose_dir
del sunflower_dir

In [ ]:
datagen = ImageDataGenerator(
        zoom_range = 0.1,
        rotation_range= 15, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,  
        vertical_flip=True)

datagen.fit(X_train)

###CNN do zero

In [ ]:
inp = Input((224,224,3))

conv1 = Conv2D(64, (5,5), padding='valid', activation= 'relu')(inp)
conv1 = MaxPooling2D(pool_size=(2,2))(conv1)
conv1 = BatchNormalization()(conv1)

conv2 = Conv2D(96, (4,4), padding='valid', activation= 'relu')(conv1)
conv2 = MaxPooling2D(pool_size=(2,2))(conv2)
conv2 = BatchNormalization()(conv2)

conv3 = Conv2D(128, (3,3), padding='valid', activation= 'relu')(conv2)
conv3 = MaxPooling2D(pool_size=(2,2))(conv3)
conv3 = BatchNormalization()(conv3)

conv4 = Conv2D(256, (3,3), padding='valid', activation= 'relu')(conv3)
conv4 = MaxPooling2D(pool_size=(2,2))(conv4)
conv4 = BatchNormalization()(conv4)
flat = Flatten()(conv4)

dense1 = Dense(512, activation= 'relu')(flat)
dense1 = Dropout(0.5)(dense1)

dense2 = Dense(64, activation= 'relu')(dense1)
dense2 = Dropout(0.1)(dense2)

out = Dense(5, activation = 'softmax')(dense2)

model = Model(inp, out)
model.compile(optimizer = Nadam(lr = 0.0001) , loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 50, initial_epoch = 0, validation_data = (X_val, y_val))

###Transfer Learning

In [ ]:
vgg = keras.applications.VGG16(input_shape=(224,224,3), include_top = False, weights= 'imagenet')

In [ ]:
x = vgg.output
x = Flatten()(x)
x = Dense(3078,activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(256,activation='relu')(x) 
x = Dropout(0.2)(x)
out = Dense(5,activation='softmax')(x)

tf_model=Model(inputs=vgg.input,outputs=out)

for layer in tf_model.layers[:20]:
    layer.trainable=False

In [ ]:
history = tf_model.fit(X_train, y_train, batch_size = 1, epochs = 30, initial_epoch = 0, validation_data = (X_val, y_val))

###Classificando alguns exemplos

In [ ]:
pred = tf_model.predict(X_val)
pred = np.argmax(pred, axis = 1)